# Tournament Structure Functions

In [1]:
import pandas as pd
import numpy as np
import random
import math

def simulate_match(player1, player2, h2h_table):
    prob_p1 = h2h_table.loc[player1, player2]

    if prob_p1 == -1:
        return "invalid_bracket"
        
    return player1 if np.random.rand() < prob_p1 else player2

def simulate_bracket_with_placings(initial_bracket, h2h_table):
    current_round = initial_bracket
    placings = {competitor: None for competitor in h2h_table.index}  # Final placings
    eliminated_this_round = []

    round_number = 1
    while len(current_round) > 0:  # Simulate rounds until we get a winner
        current_place = len(current_round) * 2
        next_round = []
        for match in current_round:
            # Simulate the match
            winner = simulate_match(match[0], match[1], h2h_table)

            if winner == "invalid_bracket":
                return None
            
            loser = match[1] if winner == match[0] else match[0]
            next_round.append(winner)
            eliminated_this_round.append(loser)
        
        # Assign placings to eliminated players (if this was their final round)
        for player in eliminated_this_round:
            if placings[player] is None:
                placings[player] = current_place # Assign based on elimination order

        # Prepare for the next round
        current_round = [(next_round[i], next_round[i + 1]) for i in range(0, len(next_round), 2)] if len(next_round) > 1 else []
        eliminated_this_round.clear()
        round_number += 1

    # Assign the winner their placing (last player standing)
    if next_round:  # Check if there's still a player remaining
        winner = next_round[0]
        placings[winner] = 1

    return placings

# Function to generate a random bracket
def generate_random_bracket(h2h_table):
    competitors = list(h2h_table.index)
    random.shuffle(competitors)  # Shuffle competitors randomly

    # Limit competitors to the largest power of 2 less than or equal to the number of players
    max_power_of_2 = 2 ** int(math.log2(len(competitors)))
    limited_competitors = competitors[:max_power_of_2]

    # Pair competitors into matches
    bracket = [(limited_competitors[i], limited_competitors[i + 1]) for i in range(0, len(limited_competitors), 2)]

    return bracket

def simulate_multiple_brackets(h2h_table, num_simulations):
    results = []

    for _ in range(num_simulations):
        # Generate a random bracket
        random_bracket = generate_random_bracket(h2h_table)
        
        # Simulate the bracket and get placings
        placings = simulate_bracket_with_placings(random_bracket, h2h_table)
        
        # If the bracket is invalid (None), skip this iteration
        if placings is None:
            continue
        
        results.append(placings)

    # Convert results into a DataFrame
    results_df = pd.DataFrame(results)  # Do not fill missing values; keep NaN for players not in brackets
    return results_df

# Function to convert the raw H2H table to win probabilities
def convert_h2h_to_probs(h2h_raw):
    h2h_probs = h2h_raw.copy()

    for row in h2h_probs.index:
        for col in h2h_probs.columns:
            raw_record = h2h_probs.loc[row, col]
            
            if raw_record == "0 - 0":
                # No matches played, set probability to 0.5 (or any other default)
                h2h_probs.loc[row, col] = -1 #0.5
            else:
                wins, losses = map(int, raw_record.split(" - "))
                total = wins + losses
                # Calculate win probability
                h2h_probs.loc[row, col] = wins / total if total > 0 else 0.5
    
    return h2h_probs.astype(float)



# Real Data

In [6]:
import pandas as pd
h2h_record = pd.read_csv("./h2h_records/nightclub_s10_top30.csv")
h2h_record = h2h_record.rename(columns={"Unnamed: 0":"player"})

bracket_size = 8

h2h_sub = h2h_record.iloc[:bracket_size , :bracket_size+1]

h2h_sub = pd.DataFrame(h2h_sub).set_index("player")
h2h_sub

,aklo,epoodle,e-tie,gl!tch,daniel,freezus,tito jojo,tazio
player,,,,,,,,
aklo,0 - 0,4 - 0,2 - 0,6 - 0,3 - 0,3 - 0,1 - 0,5 - 0
epoodle,0 - 4,0 - 0,5 - 1,1 - 1,0 - 0,2 - 2,1 - 1,2 - 0
e-tie,0 - 2,1 - 5,0 - 0,5 - 1,0 - 1,5 - 0,2 - 0,1 - 0
gl!tch,0 - 6,1 - 1,1 - 5,0 - 0,8 - 3,4 - 6,2 - 0,3 - 3
daniel,0 - 3,0 - 0,1 - 0,3 - 8,0 - 0,2 - 0,0 - 0,2 - 2
freezus,0 - 3,2 - 2,0 - 5,6 - 4,0 - 2,0 - 0,2 - 0,1 - 1
tito jojo,0 - 1,1 - 1,0 - 2,0 - 2,0 - 0,0 - 2,0 - 0,1 - 0
tazio,0 - 5,0 - 2,0 - 1,3 - 3,2 - 2,1 - 1,0 - 1,0 - 0


In [7]:
# Convert raw H2H table to probabilities
h2h_probs = convert_h2h_to_probs(h2h_sub)
h2h_probs

,aklo,epoodle,e-tie,gl!tch,daniel,freezus,tito jojo,tazio
player,,,,,,,,
aklo,-1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0
epoodle,0.0,-1.000000,0.833333,0.500000,-1.000000,0.5,0.5,1.0
e-tie,0.0,0.166667,-1.000000,0.833333,0.000000,1.0,1.0,1.0
gl!tch,0.0,0.500000,0.166667,-1.000000,0.727273,0.4,1.0,0.5
daniel,0.0,-1.000000,1.000000,0.272727,-1.000000,1.0,-1.0,0.5
freezus,0.0,0.500000,0.000000,0.600000,0.000000,-1.0,1.0,0.5
tito jojo,0.0,0.500000,0.000000,0.000000,-1.000000,0.0,-1.0,1.0
tazio,0.0,0.000000,0.000000,0.500000,0.500000,0.5,0.0,-1.0


In [8]:
# Example usage
num_simulations = 10000

# Simulate multiple brackets
simulation_results_df = simulate_multiple_brackets(h2h_probs, num_simulations)

# Calculate average placings
average_placings = simulation_results_df.mean().sort_values()

# Display results
print("\nAverage Placings:")
print(average_placings)


Average Placings:
aklo         1.000000
e-tie        5.219623
gl!tch       5.359649
epoodle      5.548083
daniel       5.716699
freezus      6.186810
tazio        6.959389
tito jojo    7.009747
dtype: float64


In [9]:
simulation_results_df.shape

(6156, 8)